We define a generic class InputGeneric which is inherited to InputVariable and InputVariablesGroup.
We detect a variable (InputVariable) by the presence of the key 'default' otherwise it is a group of variables (inputVariablesGroup).

In [1]:
from difflib import SequenceMatcher
import yaml
from io import IOBase

def closest_keys(inputs,searchfor):
    """Find the closest input variables from searchfor"""
    doc = []
    for (var,ratio) in inputs.match(searchfor):
        if ratio >= 0.5:
            doc.append(var)
    return doc
    #print yaml.dump(doc, default_flow_style=False, explicit_start=True)
    
class FindKeys(list):
    """List of InputVariables matching a sequence."""
    #def __new__(cls,:
    #    return list.__new__(cls) 
    def __init__(self,inputs,searchfor):
        self += closest_keys(inputs,searchfor)
        self.searchfor=searchfor
    def __str__(self):
        string='Keys which match the most with "%s"\n' % self.searchfor
        string+=str([ (c.name,c.COMMENT) for c in self])
        return string
    
class InputGeneric():
    """Generic class to define input variables or group of input variables."""
    def __init__(self,name,definition,parent=None):
        """From the dictionary import the tree of variables."""
        self.name = name
        self.parent = parent
        self.__rawdict__ = definition.copy()
        self.DESCRIPTION = self.__rawdict__.pop('DESCRIPTION','')
        self.__vars__ = list(self.__rawdict__.keys())
        self.__vars__.sort()
        #By default check the last level of input variables group.
        self.__safeload = False
    def __str__(self):
        """Return the full name of the variable with its parents"""
        if self.parent:
            return str(self.parent)+"."+self.name
        else:
            return self.name
    def full(self):
        """Return a full input yaml document"""
        return yaml.dump(self.todict(),default_flow_style=False)
    def load(self,string,safe=True):
        """Load a yaml dict and set the corresponding variables.
           If the option 'safe' is false, do not check if an input variable does exist."""
        if isinstance(string,str) or isinstance(string,IOBase):
            doc = yaml.load(string, Loader = yaml.loader.SafeLoader)
        elif isinstance(string,dict):
            doc = string
        self.load_doc(doc,safe)
    def load_doc(self,doc,safe=True):
        """Load a yaml dict and set the corresponding variables.
           If the option 'safe' is false, do not check if an input variable does exist."""
        for (key,val) in doc.items():
            if key in self.__vars__:
                var = getattr(self,key)
                var.load_doc(val)
            elif safe and self.__safeload:
                raise KeyError("The key '%s' for '%s' does not exist!" % (key,self.name))
    def match(self,searchfor):
        """Compare a sequence to all keys"""
        matching_keys = []
        for k in self.__vars__:
            var = getattr(self,k)
            m = var.match(searchfor)
            if (isinstance(m,list)):
                matching_keys.extend(m)
            else:
                matching_keys.append(m)
        matching_keys.sort(key=lambda x: x[1])
        matching_keys.reverse()
        return matching_keys
    def minimal(self):
        """Return a minimal input yaml document"""
        return yaml.dump(self.tomin(),default_flow_style=False)
    def search(self,sequence):
        """Return a list (class FindKeys) of Inputvariables matching the sequence."""
        return FindKeys(self,sequence)
    def todict(self):
        """Translate into a dictionary"""
        return {k: getattr(self,k).todict() for k in self.__vars__}
    def tomin(self):
        """Return a minimal input yaml document"""
        doc = {k: getattr(self,k).tomin() for k in self.__vars__}
        for (key,val) in list(doc.items()):
            if val == 'default' or val == {}:
                del doc[key]
        return doc
    def wiki(self):
        """Wiki representation of the input variables"""
        cr="\n"
        string="="+self.name+"="+cr
        for var in self.__vars__:
            variable=getattr(self,var)
            string+=wiki(variable)
        return string
    
class InputVariablesGroup(InputGeneric):
    """Define group of input variables."""
    def __init__(self,name,definition,parent=None):
        InputGeneric.__init__(self,name,definition,parent)
        for var in self.__rawdict__:
            if 'default' in definition[var]:
                setattr(self,var,InputVariable(var,definition[var],self))
                #Last level of groups of input variables
                self.__safeload = True
            else:
                setattr(self,var,InputVariablesGroup(var,definition[var],self))
    
class InputVariable(InputGeneric):
    """Class of input variable"""
    def __init__(self,name,definition,parent=None):
        """Define the variable associated to the input variable in futile proposed format"""
        #Raw definiton
        InputGeneric.__init__(self,name,definition,parent)
        #Set the attribute and the meta values
        self.__meta = []
        for key in definition:
            setattr(self,key,definition[key])
            if key not in ['COMMENT', 'DESCRIPTION', 'default', 'CONDITION', 'EXCLUSIVE', 'RANGE']:
                self.__meta.append(key)
        #Set the default value (use 'default' to avoid the condition)
        self.set('default')
    def __str__(self):
        """return the full name and the value of the variable"""
        return InputGeneric.__str__(self)+": "+str(self.value)
    def load_doc(self,doc):
        """Set the value from a document"""
        self.set(doc)
    def match(self,searchfor):
        """Give the match of the input variable with a given sequence"""
        #s = SequenceMatcher(lambda x: x == " ", searchfor, 
        #                    "%s %s %s" % (self.name,self.COMMENT,self.DESCRIPTION))
        slow=searchfor.lower()
        if slow in self.COMMENT.lower():
            ratio = 1.0
        elif slow in self.DESCRIPTION.lower():
            ratio = 1.0
        else:
            ratio = SequenceMatcher(lambda x: x == " ", slow,self.name.lower()).ratio()
        return (self, ratio)
    def set(self,value,reset=True):
        """Set the value for the corresponding variables and check it."""
        #Special case for default
        if value == 'default': 
            self.value = self.default
            #Check nothing (not useful and avoid CONDITION the first time)
            return
        else:
            self.value = value
        #Check if the value is equal to a meta value and replace by the meta key
        for m in self.__meta:
            if self.value == getattr(self,m):
                self.value = m
        if self.value in self.__meta:
            pass
        #here we can check that the value is legal following the definitions
        elif hasattr(self,'EXCLUSIVE') and self.value not in self.EXCLUSIVE:
            raise TypeError('Permitted value: '+str(self.EXCLUSIVE)+ \
                            "\nValue ('"+str(self.value)+"') not admitted for variable "+str(self.name))
        elif hasattr(self,'RANGE') and (self.value < self.RANGE[0] or self.value > self.RANGE[1]):
            raise TypeError('Permitted range: '+str(self.RANGE)+ \
                            "\nValue ("+str(self.value)+") not admitted for variable "+str(self.name))
        #We check the CONDITION 
        if hasattr(self,'CONDITION'):
            cond = getattr(self,'CONDITION')
            master = getattr(self.parent,cond['MASTER_KEY'])
            #Put to default if the condition is not satisfied
            if 'WHEN' in cond and (not master.value in cond['WHEN']):
                self.value = self.default
            elif 'WHEN_NOT' in cond and (master.value in cond['WHEN_NOT']):
                self.value = self.default
        if reset: 
            self.re_set()
    
    def re_set(self):
        """Re-set all variables of the parent"""
        parent=self.parent
        for key in parent.__vars__:
            if key==self.name: continue
            var=getattr(parent,key)
            var.set(var.value,reset=False)
    
    def todict(self):
        """Return the value"""
        return self.value
    def tomin(self):
        """Return the value if it is not the default"""
        if self.value == self.default:
            return 'default'
        else:
            return self.value
    def wiki(self):
        cr="\n"
        string="=="+self.name+"=="+cr
        if hasattr(self,'DESCRIPTION'):
            string+=self.DESCRIPTION+cr
        elif hasattr(self,'COMMENT'):
            string+=self.COMMENT+cr
        string+="Default value: "+str(self.default)+cr
        if hasattr(self,'EXCLUSIVE'):
            for ex in self.EXCLUSIVE:
                string+='* '+str(ex)+cr
        return string


In [2]:
import yaml
isf=yaml.load('''
  isf_order: 
    COMMENT: Order of the Interpolating Scaling Function family
    DESCRIPTION: Fixes the order of the ISF family that is used for the discretization of the kernel
    default: 16
    EXCLUSIVE: [ 2, 4, 6, 8, 14, 16, 20, 24, 30, 40, 50, 60, 100]
''', Loader = yaml.loader.SafeLoader)
print(isf)

isfvar=InputVariable('isf_order',isf['isf_order'])
print(isfvar.wiki())
print(isfvar.todict())
a = isfvar.match('isf')
print(a[0].name,a[1])

{'isf_order': {'COMMENT': 'Order of the Interpolating Scaling Function family', 'DESCRIPTION': 'Fixes the order of the ISF family that is used for the discretization of the kernel', 'default': 16, 'EXCLUSIVE': [2, 4, 6, 8, 14, 16, 20, 24, 30, 40, 50, 60, 100]}}
==isf_order==
Fixes the order of the ISF family that is used for the discretization of the kernel
Default value: 16
* 2
* 4
* 6
* 8
* 14
* 16
* 20
* 24
* 30
* 40
* 50
* 60
* 100

16
isf_order 1.0


In [3]:
docs=[a for a in yaml.load_all(open('test_input_variables_definition.yaml','r'), Loader = yaml.loader.SafeLoader)]
ps=docs[0]
psolver = InputVariablesGroup('psolver',ps)
psolver.environment.epsilon.set(78.36)
print(psolver.environment.epsilon.value == 78.36)
print(psolver.environment.epsilon.value == 'water')

psolver.load(open('test-input.yaml','r'))

print(psolver.minimal())

False
True
environment:
  itermax: 0
setup:
  taskgroup_size: 1



In [4]:
cc = psolver.search('min')
print(cc)
var = cc[0]
print(var, var.default,var.tomin())
var.set(4.0)
print(var.parent.parent)
print(psolver.minimal())

Keys which match the most with "min"
[('minres', 'Convergence threshold of the loop'), ('pb_minres', 'Convergence criterion of the PBe loop')]
psolver.environment.minres: 1e-08 1e-08 default
psolver
environment:
  itermax: 0
  minres: 4.0
setup:
  taskgroup_size: 1



In [5]:
psolver.environment.epsilon.set(1.1)
psolver.environment.epsilon.todict()
psolver.environment.epsilon.todict()

'water'

In [6]:
psolver.setup.accel.set('CUDA')
psolver.setup.use_gpu_direct.set(False)
print(psolver.minimal())
print(30*'=')
psolver.setup.accel.set('none')
print(psolver.minimal())
print(30*'=')
psolver.setup.use_gpu_direct.set(True)
print(psolver.setup.use_gpu_direct.CONDITION)
print(psolver.minimal())

environment:
  itermax: 0
  minres: 4.0
setup:
  accel: CUDA
  taskgroup_size: 1
  use_gpu_direct: false

environment:
  itermax: 0
  minres: 4.0
setup:
  taskgroup_size: 1

{'MASTER_KEY': 'accel', 'WHEN': ['CUDA']}
environment:
  itermax: 0
  minres: 4.0
setup:
  taskgroup_size: 1

